# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/natalymartinez/Desktop/Module Challenges/Module 6 Challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display
# change the data type of Lng and Lat to float 
city_data_df["Lat"] = city_data_df["Lat"].astype(float)
city_data_df["Lng"] = city_data_df["Lng"].astype(float)

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = .5,
    color = 'City'
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = 27
min_temp = 21
max_wind_speed = 10
max_cloudiness = 0
filtered_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] < max_temp) &
                                     (city_data_df['Max Temp'] > min_temp) &
                                     (city_data_df['Wind Speed'] < max_wind_speed) &
                                     (city_data_df['Cloudiness'] <= max_cloudiness)
                                     ]


# Drop any rows with null values
filtered_city_data_df = city_data_df.dropna()

# Display sample data
filtered_city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228,
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230,
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934,
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230,
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231,
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231,
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232,
8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235,
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = { "categories": categories,
          "limit": limit,
            "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionary & convert to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
colorado - nearest hotel: No hotel found
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: No hotel found
dobryanka - nearest hotel: No hotel found
qaanaaq - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: No hotel found
tasiilaq - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
nuevo laredo - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
rikitea - nearest hotel: No hot

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228,No hotel found
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228,No hotel found
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230,No hotel found
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934,No hotel found
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564,No hotel found
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564,No hotel found
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565,No hotel found
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

%%capture --no-display

# Configure the map plot
hotel_plot_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 800, frame_height = 600, scale = 0.8, color="City", hover_cols=['City', 'Hotel Name', 'Country'])

# Display the map
hotel_plot_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)